### This notebook parses the attributed lives into patient panels by Organization 
##### Added Excel Automation including Filtering, Styling, Security, Pre-filtering 
#### Joe Eberle - 2/22/2022 Updated on 4/7/2022 

In [ ]:
## -- Current Attribution 
## select * from DS_ATTRIB_AFFILIA_PATIENT_DETAIL

In [1]:
# Import Libraries for Spreadsheet Automation 
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date
import random
import math
import os
import glob
from openpyxl import load_workbook
from openpyxl.styles import colors
from openpyxl.utils import get_column_letter
from openpyxl.styles import NamedStyle, Font, Border, Side, PatternFill, GradientFill, Alignment, Color, Fill
from openpyxl import Workbook
import openpyxl
import time

In [13]:
# Configure the code and visual settings to have consistency 
# path = 'c:/Users/josep/Documents/Kaleida/Attribution/'
#Path_to_Excel = 'c:/Users/josep/Documents/Kaleida/Attribution/'
#Path_to_Save_Excel = 'c:/Users/josep/Documents/Kaleida/Attribution_Formated/'

Path_to_Excel = 'J:/IT GLIN Data Services Shared/Kaleida/Attributed_Patient_Panel/'
Path_to_Save_Excel = 'J:/IT GLIN Data Services Shared/Kaleida/Attributed_Patient_Panel_ft/'

path = 'J:/IT GLIN Data Services Shared/Kaleida/'
# Path_to_Share_Excel =  'https://kaleidahealth.sharefile.com/fo54e2c6-ffc7-4025-8649-443d1d382dff//'
# Path_to_Raw_Data = 'c:/Users/josep/Documents/Kaleida/'
Path_to_Raw_Data = 'J:/IT GLIN Data Services Shared/Kaleida/'
Raw_Data_File_Name_1 = 'Attributed Patient Panel.xlsx'
Raw_Data_File_Name_2 = 'Organization_Provider_Summary.xlsx'
Raw_Data_File_Name_3 = 'Organization_Summary.xlsx'

print(Path_to_Raw_Data + Raw_Data_File_Name_1)
print(Path_to_Raw_Data + Raw_Data_File_Name_2)
print(Path_to_Raw_Data + Raw_Data_File_Name_3)

LIGHT_BLUE = "C5D9F1"
YELLOW = "FFFF00"
GRAY = "E4DFEC"
DEEP_BLUE = "1072BA"
WHITE = "FFFFFF"

CONTENT_TAB_COLOR = DEEP_BLUE
USER_TAB_COLOR = YELLOW
BLANK_WHITE = WHITE 
HEADER_COLOR = LIGHT_BLUE  # Light Blue - Light Blue indicates relevant content for validation 
HEADER_COLOR_USER_ENTRY = YELLOW # Yellow - Yellow indicates a field for User Entry 
HEADER_COLOR_INFORMATIONAL_ONLY = GRAY # Gray - Indicates a field that maybe helpful but for information purposes only 

J:/IT GLIN Data Services Shared/Kaleida/Attributed Patient Panel.xlsx
J:/IT GLIN Data Services Shared/Kaleida/Organization_Provider_Summary.xlsx
J:/IT GLIN Data Services Shared/Kaleida/Organization_Summary.xlsx


In [14]:
# Read in the CSV SQL output for Patient Attribution 
patient_panel = pd.read_excel(Path_to_Raw_Data + Raw_Data_File_Name_1)

In [15]:
# Read in the EXCEL  SQL output for Organization Provider Summary 
Organization_Provider_Summary = pd.read_excel(Path_to_Raw_Data + Raw_Data_File_Name_2)

In [16]:
# Read in the EXCEL  SQL output for Organization Provider Summary 
Organization_Summary = pd.read_excel(Path_to_Raw_Data + Raw_Data_File_Name_3)

In [ ]:
# Optional Verify the input file structures 
#patient_panel.info()
#Organization_Provider_Summary.head()
#patient_panel.columns

In [17]:
# Trim Time from Birth Date and Add Additional fields to support User Validation 
patient_panel['Birth_Date'] = patient_panel['Birth_Date'].dt.date 
patient_panel['Is Patient cared for by Organization (Y/N) ?'] = ' '
patient_panel['Is Patient attributed to correct PCP (Y/N) ?'] = ' '
patient_panel['Optional Comments'] = ' '
patient_panel['Flagged for Test (Y/N)'] = 'N'
patient_panel['Is Patient info Correct? (Y/N)'] = ' '

In [18]:
# simplify and Organize the patient Panels
patient_panel = patient_panel[['Provider_Name','Patient_Name', 'Birth_Date', 'Age',
                               'Gender', 'Phone','Is Patient cared for by Organization (Y/N) ?', 
                               'Is Patient attributed to correct PCP (Y/N) ?',
                               'Optional Comments','Deceased', 'Deceased_Date','Flagged for Test (Y/N)',
                               'Organization_Class', 'Organization_Name',
                               'Plan_Name', 'Payer_Name', 'Age_Years','Is Patient info Correct? (Y/N)',]]

In [21]:
# The following function creates patient panels based upon organization name 
def create_organization_patient_panel(org_name):
    Organization_panel = patient_panel[(patient_panel.Organization_Name == org_name)]    
    patient_rows = len(Organization_panel) - 1 
    if patient_rows > 2:
        spread_name = Organization_panel['Organization_Name'].iloc[0]
        csv_name = Organization_panel['Organization_Name'].iloc[0]
        if spread_name == '':
            spread_name = 'No_ORG_NAME_' 
        if csv_name == '':
            csv_name = 'No_ORG_NAME_'              
        spread_name = spread_name.replace('/','')  
        csv_name = csv_name.replace('/','')          
        lesser = min(patient_rows,25)
        for x in range(lesser):
            Organization_panel.iat[random.randint(0,patient_rows),11] = 'Y'
       # spath = 'c:/Users/josep/Documents/Kaleida/Validation/'
        spath = Path_to_Excel
        spread_name = spath + spread_name.replace(' ','_') + '.xlsx'
        csv_name = spath + csv_name.replace(' ','_') + '.csv'
        #spread_name = spread_name.replace(' ','')
        #csv_name = csv_name.replace(' ','')        
        Organization_panel.to_excel(spread_name,sheet_name='Patient Attribution', index = False)  
        Organization_panel.to_csv(csv_name, index = False)          
        print(spread_name)
        return org_name

In [22]:
Org_names = patient_panel.Organization_Name.unique()
for org in Org_names:
    create_organization_patient_panel(org)

J:/IT GLIN Data Services Shared/Kaleida/Attributed_Patient_Panel/EC_PCP_Erie_County_Medical_Center_FM_Clinic.xlsx
J:/IT GLIN Data Services Shared/Kaleida/Attributed_Patient_Panel/EC_PCP_Erie_County_Medical_Center_Grider_Fam_Health_Clinic.xlsx
J:/IT GLIN Data Services Shared/Kaleida/Attributed_Patient_Panel/EC_PCP_Erie_County_Medical_Center_IM_Clinic.xlsx
J:/IT GLIN Data Services Shared/Kaleida/Attributed_Patient_Panel/EC_PCP_Erie_County_Medical_Center_VIP_Clinic.xlsx
J:/IT GLIN Data Services Shared/Kaleida/Attributed_Patient_Panel/EC_PCP_Erie_County_Medical_Center_You_Center_for_Wellness.xlsx
J:/IT GLIN Data Services Shared/Kaleida/Attributed_Patient_Panel/Erie_County_Medical_Center_Corporation.xlsx
J:/IT GLIN Data Services Shared/Kaleida/Attributed_Patient_Panel/Great_Lakes_Physician_PC.xlsx
J:/IT GLIN Data Services Shared/Kaleida/Attributed_Patient_Panel/GPPC_PCP_Amherst_Sheridan.xlsx
J:/IT GLIN Data Services Shared/Kaleida/Attributed_Patient_Panel/GPPC_PCP_Buffalo_Main.xlsx
J:/IT GL

J:/IT GLIN Data Services Shared/Kaleida/Attributed_Patient_Panel/WN_PCP_Western_NY_Med_1310_French.xlsx
J:/IT GLIN Data Services Shared/Kaleida/Attributed_Patient_Panel/WN_PCP_Western_NY_Med_3218_Walden.xlsx
J:/IT GLIN Data Services Shared/Kaleida/Attributed_Patient_Panel/WN_PCP_Western_NY_Med_3320_N.xlsx
J:/IT GLIN Data Services Shared/Kaleida/Attributed_Patient_Panel/WN_PCP_Western_NY_Med_3595_Eggert.xlsx
J:/IT GLIN Data Services Shared/Kaleida/Attributed_Patient_Panel/WN_PCP_Western_NY_Med_3884_Broadway.xlsx
J:/IT GLIN Data Services Shared/Kaleida/Attributed_Patient_Panel/WN_PCP_Western_NY_Med_4247_Maple.xlsx
J:/IT GLIN Data Services Shared/Kaleida/Attributed_Patient_Panel/WN_PCP_Western_NY_Med_6970_Erie.xlsx
J:/IT GLIN Data Services Shared/Kaleida/Attributed_Patient_Panel/YP_PCP_Yasmin_Pervez,_MD_Wheatfield_St.xlsx
J:/IT GLIN Data Services Shared/Kaleida/Attributed_Patient_Panel/Independent_Providers.xlsx
J:/IT GLIN Data Services Shared/Kaleida/Attributed_Patient_Panel/Limestone_Pr

In [26]:
# Add the Organization Summary Excel Tab  
extension = 'xlsx'
os.chdir(Path_to_Excel)
files = glob.glob('*.{}'.format(extension))
 
for x in range(len(files)): 
#    print(files[x]) 
    file_path = Path_to_Excel + files[x]
    org_name =  files[x].replace('.xlsx','')
    org_name =  org_name.replace('_',' ')    
    print (org_name)
    Org_Summary = Organization_Summary[(Organization_Summary.Organization_Name == org_name)]  
    Org_Summary = Org_Summary.reset_index(drop=True)
  
    writer = pd.ExcelWriter(file_path, engine='openpyxl', mode='a')
    writer.book = load_workbook(file_path)
    Org_Summary.to_excel(writer,sheet_name='Ogranization Summary', index = False)
    writer.save()
    writer.close()

EC PCP Erie County Medical Center FM Clinic
EC PCP Erie County Medical Center Grider Fam Health Clinic
EC PCP Erie County Medical Center IM Clinic
EC PCP Erie County Medical Center VIP Clinic
EC PCP Erie County Medical Center You Center for Wellness
Erie County Medical Center Corporation
Great Lakes Physician PC
GPPC PCP Amherst Sheridan
GPPC PCP Buffalo Main
GPPC PCP Dunkirk Vineyard Ste 1
GPPC PCP East Aurora Main
GPPC PCP Elma Transit
GPPC PCP Hamburg Camp
GPPC PCP Hamburg Southwestern
GPPC PCP Lancaster Transit
GPPC PCP Lockport Olcott
GPPC PCP Lockport Snyder
GPPC PCP Medina Ohio
GPPC PCP Orchard Park Big Tree Ste 107
GPPC PCP Springville South Cascade
GPPC PCP Tonawanda Colvin
GPPC PCP Willamsville Maple
GPPC PCP Williamsville Youngs
GPPC Peds Dunkirk Vineyard Ste 3
GPPC Post-Acute
General Physician PC
AA PCP Alan M Antfleck Physician PLLC Delaware Ave
AFC PCP Akron Family Care 16 Main St
AP PCP Amandeep Pal MD PLLC Robinson Road
BF PCP Bradford Family Medicine 14 N Third St
BRM 

In [27]:
# Add the Provider Summary Excel Tab  
extension = 'xlsx'
os.chdir(Path_to_Excel)
print('Path = '+ path)
files = glob.glob('*.{}'.format(extension))
 

for x in range(len(files)): 
#    print(files[x]) 
    file_path = Path_to_Excel + files[x]
    org_name =  files[x].replace('.xlsx','')
    org_name =  org_name.replace('_',' ')    
    print (org_name)
    Provider_Summary = Organization_Provider_Summary[(Organization_Provider_Summary.Organization_Name == org_name)]  
    Provider_Summary = Provider_Summary.reset_index(drop=True)
  
    writer = pd.ExcelWriter(file_path, engine='openpyxl', mode='a')
    writer.book = load_workbook(file_path)
    Provider_Summary.to_excel(writer,sheet_name='Provider Summary', index = False)
    writer.save()
    writer.close()

Path = J:/IT GLIN Data Services Shared/Kaleida/
EC PCP Erie County Medical Center FM Clinic
EC PCP Erie County Medical Center Grider Fam Health Clinic
EC PCP Erie County Medical Center IM Clinic
EC PCP Erie County Medical Center VIP Clinic
EC PCP Erie County Medical Center You Center for Wellness
Erie County Medical Center Corporation
Great Lakes Physician PC
GPPC PCP Amherst Sheridan
GPPC PCP Buffalo Main
GPPC PCP Dunkirk Vineyard Ste 1
GPPC PCP East Aurora Main
GPPC PCP Elma Transit
GPPC PCP Hamburg Camp
GPPC PCP Hamburg Southwestern
GPPC PCP Lancaster Transit
GPPC PCP Lockport Olcott
GPPC PCP Lockport Snyder
GPPC PCP Medina Ohio
GPPC PCP Orchard Park Big Tree Ste 107
GPPC PCP Springville South Cascade
GPPC PCP Tonawanda Colvin
GPPC PCP Willamsville Maple
GPPC PCP Williamsville Youngs
GPPC Peds Dunkirk Vineyard Ste 3
GPPC Post-Acute
General Physician PC
AA PCP Alan M Antfleck Physician PLLC Delaware Ave
AFC PCP Akron Family Care 16 Main St
AP PCP Amandeep Pal MD PLLC Robinson Road
BF

In [28]:
# The following function creates patient panels based upon organization name 
def create_sheet_one(work_book):
    
    ws1 = work_book.active
    ws1.title = "Patient Attribution"
    ws1.sheet_properties.tabColor = CONTENT_TAB_COLOR 
    
# Format the Header as text bold and backgound light blue centered wordwrap     
    max_cols = ws1.max_column + 1 
    for col in range(1,max_cols):
        ws1[get_column_letter(col) + '1'].font = Font(bold=True ) 
        ws1[get_column_letter(col) + '1'].fill = PatternFill("solid", start_color=HEADER_COLOR) 
        ws1[get_column_letter(col) + '1'].alignment = Alignment(horizontal='center', wrap_text=True)  
        ws1[get_column_letter(col) + '1'].value = ws1[get_column_letter(col) + '1'].value.replace('_','  ')

# Assign special Yellow column headers for User Entry 
    ws1['G1'].fill = PatternFill("solid", start_color=HEADER_COLOR_USER_ENTRY)                
    ws1['H1'].fill = PatternFill("solid", start_color=HEADER_COLOR_USER_ENTRY) 
    ws1['I1'].fill = PatternFill("solid", start_color=HEADER_COLOR_USER_ENTRY)   
    
# Assign special Gray columns for Informational Purposes ONLY    
    
    ws1['J1'].fill = PatternFill("solid", start_color=HEADER_COLOR_INFORMATIONAL_ONLY)   
    ws1['K1'].fill = PatternFill("solid", start_color=HEADER_COLOR_INFORMATIONAL_ONLY)   
    ws1['L1'].fill = PatternFill("solid", start_color=HEADER_COLOR_INFORMATIONAL_ONLY)   
    ws1['M1'].fill = PatternFill("solid", start_color=HEADER_COLOR_INFORMATIONAL_ONLY) 
    ws1['N1'].fill = PatternFill("solid", start_color=HEADER_COLOR_INFORMATIONAL_ONLY)   
    ws1['O1'].fill = PatternFill("solid", start_color=HEADER_COLOR_INFORMATIONAL_ONLY)   
    ws1['P1'].fill = PatternFill("solid", start_color=HEADER_COLOR_INFORMATIONAL_ONLY)   
    ws1['Q1'].fill = PatternFill("solid", start_color=HEADER_COLOR_INFORMATIONAL_ONLY)
    ws1['R1'].fill = PatternFill("solid", start_color=HEADER_COLOR_INFORMATIONAL_ONLY)

    
# Work Sheet 1 - Patient Attribution - Set Column Widths and Header Hieght
    ws1.column_dimensions['A'].width = 25
    ws1.column_dimensions['B'].width = 25
    ws1.column_dimensions['C'].width = 14
    ws1.column_dimensions['D'].width = 20
    ws1.column_dimensions['E'].width = 15
    ws1.column_dimensions['F'].width = 15
    ws1.column_dimensions['G'].width = 10
    ws1.column_dimensions['H'].width = 10
    ws1.column_dimensions['I'].width = 20
    ws1.column_dimensions['J'].width = 10
    ws1.column_dimensions['K'].width = 10
    ws1.column_dimensions['L'].width = 10
    ws1.column_dimensions['M'].width = 25
    ws1.column_dimensions['N'].width = 25
    ws1.column_dimensions['O'].width = 25
    ws1.column_dimensions['P'].width = 25
    ws1.column_dimensions['Q'].width = 25    
    ws1.row_dimensions[1].height = 75   

    
# Set Filtering on 
    FullRange = "A1:" + get_column_letter(ws1.max_column) \
    + str(ws1.max_row)
    ws1.auto_filter.ref = FullRange    

    return work_book  

In [29]:
# The following function formats provider summary 
def create_sheet_two(work_book):
    
    sheets = work_book.sheetnames
    ws2 = work_book[sheets[1]]
    ws2.title = "Organization Summary"
    ws2.sheet_properties.tabColor = CONTENT_TAB_COLOR
     
# Format the Header as bold blue centered wordwrap     
    max_cols = ws2.max_column + 1 
    for col in range(1,max_cols):
        ws2[get_column_letter(col) + '1'].font = Font(bold=True ) 
        ws2[get_column_letter(col) + '1'].fill = PatternFill("solid", start_color=HEADER_COLOR) 
        ws2[get_column_letter(col) + '1'].alignment = Alignment(horizontal='center', wrap_text=True)    
    
# Work Sheet 1 - Patient Attribution - Set Column Widths and Header Hieght
    ws2.column_dimensions['A'].width = 30
    ws2.column_dimensions['B'].width = 30
    ws2.column_dimensions['C'].width = 30
    ws2.column_dimensions['D'].width = 30
    ws2.column_dimensions['E'].width = 30
    ws2.column_dimensions['F'].width = 30

    ws2.row_dimensions[1].height = 60    

# Replaces all underscores in header with two spaces 
    for col in range(1,9):
        ws2[get_column_letter(col) + '1'].value = ws2[get_column_letter(col) + '1'].value.replace('_','  ')
    
# Set Filtering on 
    FullRange = "A1:" + get_column_letter(ws2.max_column) \
    + str(ws2.max_row)
    ws2.auto_filter.ref = FullRange    


    return work_book  

In [30]:
# The following function formats provider summary 
def create_sheet_three(work_book):
    
    sheets = work_book.sheetnames
    ws3 = work_book[sheets[2]]
    ws3.title = "Provider Summary"
    ws3.sheet_properties.tabColor = CONTENT_TAB_COLOR
     
# Format the Header as bold blue centered wordwrap     
    max_cols = ws3.max_column + 1 
    for col in range(1,max_cols):
        ws3[get_column_letter(col) + '1'].font = Font(bold=True ) 
        ws3[get_column_letter(col) + '1'].fill = PatternFill("solid", start_color=HEADER_COLOR) 
        ws3[get_column_letter(col) + '1'].alignment = Alignment(horizontal='center', wrap_text=True)    
    
# Work Sheet 1 - Patient Attribution - Set Column Widths and Header Hieght
    ws3.column_dimensions['A'].width = 30
    ws3.column_dimensions['B'].width = 30
    ws3.column_dimensions['C'].width = 30
    ws3.column_dimensions['D'].width = 30
    ws3.column_dimensions['E'].width = 30
    ws3.column_dimensions['F'].width = 30

    ws3.row_dimensions[1].height = 60    

# Replaces all underscores in header with two spaces 
    for col in range(1,9):
        ws3[get_column_letter(col) + '1'].value = ws3[get_column_letter(col) + '1'].value.replace('_','  ')
    
# Set Filtering on 
    FullRange = "A1:" + get_column_letter(ws3.max_column) \
    + str(ws3.max_row)
    ws3.auto_filter.ref = FullRange    


    return work_book  

In [31]:
def create_sheet_validation(work_book):
    
    wsval = work_book.create_sheet("Validation Summary")
    wsval.title = "Validation Summary" 
    wsval.sheet_properties.tabColor = USER_TAB_COLOR
    
    wsval.merge_cells('A1:F1')
    wsval.row_dimensions[1].height = 20
    top_left_cell = wsval['A1']
    top_left_cell.value = "Validation Summary"

    top_left_cell.fill = PatternFill("solid", fgColor=USER_TAB_COLOR)
    top_left_cell.fill = fill = GradientFill(stop=(USER_TAB_COLOR, BLANK_WHITE))
    top_left_cell.font  = Font(b=True, color="000000")
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    fontStyle = Font(size = "18", bold = "True")
    top_left_cell.font = fontStyle

    wsval['A3'] = 'Number of Patients to Validate:'
    wsval['A4'] = 'Number of Patients Validate as Accurate:'
    wsval['A5'] = 'Number of Patients That are Not Validate:'
    wsval['A6'] = 'Total Validated :' 
    wsval['A7'] = 'Remaining :'
    wsval['A8'] = '% of Validated as Accurate:'

    wsval['A10'] = 'Total Distinct Providers:'
    wsval['A11'] = 'Total Patient Panel Size :'
    wsval.column_dimensions['A'].width = 50


    wsval['B3'] = '=COUNTIF(\'Patient Attribution\'!L2:L50000,"Y")'
    wsval['B4'] = '=COUNTIF(\'Patient Attribution\'!G2:G50000,"Y")'
    wsval['B5'] = '=COUNTIF(\'Patient Attribution\'!G2:G50000,"N")'
    wsval['B6'] = '=B4+B5'
    wsval['B7'] = '=B3-B6'
    wsval['B8'] = '=B4/B6'
    wsval['B8'].number_format = '0.0%'

    wsval['B10'] = '=COUNTA(\'Provider Summary\'!C2:C50000)'    
    wsval['B11'] = '=COUNTA(\'Patient Attribution\'!B2:B50000)'
    return work_book 


In [32]:
def create_instruction_sheet(work_book):
    
    ws_instruction  = work_book.create_sheet("Instructions")
    ws_instruction.title = "Instructions" 
    ws_instruction.sheet_properties.tabColor = USER_TAB_COLOR
    
    # Blank the Canvas 
    fontStyle = Font(size = "12", bold = True)
    for col in range(1,25):
        for row in range(1,25):
            ws_instruction[get_column_letter(col) + str(row)].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)          
            ws_instruction[get_column_letter(col) + str(row)].font = fontStyle
            ws_instruction[get_column_letter(col) + str(row)].fill = PatternFill("solid", fgColor=BLANK_WHITE)
            
    for row in range(1,25):
            ws_instruction.row_dimensions[row].height = 20         
            
    ws_instruction.column_dimensions['A'].width = 3
    ws_instruction.column_dimensions['B'].width = 3
    ws_instruction.column_dimensions['C'].width = 80            
    
    
    ws_instruction.merge_cells('A1:F1')
    top_left_cell = ws_instruction['A1']
    top_left_cell.value = "Instructions for Validation"

    top_left_cell.fill = PatternFill("solid", fgColor=USER_TAB_COLOR)
    top_left_cell.fill = fill = GradientFill(stop=(USER_TAB_COLOR, BLANK_WHITE))
    top_left_cell.font  = Font(b=True, color="000000")
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    fontStyle = Font(size = "18", bold = "True")
    top_left_cell.font = fontStyle

    fontStyle = Font(size = "14", bold = "True")
    ws_instruction.merge_cells('A3:F3')
    ws_instruction['A3'].font = fontStyle
    ws_instruction['A3'] = 'Spreadsheet instructions:'

    ws_instruction['B4'] = 1
    ws_instruction['B5'] = 2 
    ws_instruction['B6'] = 3
    ws_instruction['B7'] = 4
    ws_instruction['B8'] = 5
    ws_instruction['B9'] = 6
    ws_instruction['B10'] = 7    
    ws_instruction['B11'] = 8      
    
    ws_instruction['C4'] = 'Please Filter column \'Flagged for Test\' to yes... it will randomly select 25 patients'
    ws_instruction['C5'] = 'Validation needs to be completed on the pre-selected patients and returned to the sender you received them from by February 22, 2022.'
    ws_instruction['C6'] = 'Please validate the data in the blue columns on the Patient Panel spreadsheet.'
    ws_instruction['C7'] = 'Enter a Y or N in the yellow column labeled \'Is Patient cared for by Organization ?\' '  
    ws_instruction['C8'] = 'Enter a Y or N in the yellow column labeled \'Is Patient attributed to correct PCP ?\' ' 
    ws_instruction['C9'] = 'If a N is entered, indicate why in the yellow column labeled  \'Optional Comments \' '
    ws_instruction['C10'] = 'Please review provider summary panel to ensure providers are part of organization. '     
    ws_instruction['C11'] = 'After validation is complete add or modify missing providers on the \'new provider\' tab. '   
    ws_instruction['C12'] = 'If any issues or problems occured please enter them on the \'Feedback\' tab. '       

    
    return work_book  


In [33]:
def create_sheet_add(work_book):
    
    wsadd = work_book.create_sheet("Adding new Physicians")
    wsadd.title = "Adding new Physicians" 
    wsadd.sheet_properties.tabColor = USER_TAB_COLOR
    
    wsadd.merge_cells('A1:G1')
    wsadd.row_dimensions[1].height = 20
    top_left_cell = wsadd['A1']
    top_left_cell.value = "Adding, Updating, or Removing Physicians"

    top_left_cell.fill = PatternFill("solid", fgColor=USER_TAB_COLOR)
    top_left_cell.fill = fill = GradientFill(stop=(USER_TAB_COLOR, BLANK_WHITE))
    top_left_cell.font  = Font(b=True, color="000000")
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    fontStyle = Font(size = "18", bold = "True")
    top_left_cell.font = fontStyle

    wsadd['A2'] = 'Provider Name'
    wsadd['B2'] = 'Provider NPI'
    wsadd['C2'] = 'Provider Credentials'
    wsadd['D2'] = 'Is Provider a Primary Care Physician PCP (Y/N) ?'
    wsadd['E2'] = 'Is Provider a Specialist (Y/N) ? '
    wsadd['F2'] = 'If Specialist enter Provider Specialty'
    wsadd['G2'] = 'Does the Provider need to be Added or Removed or Updated (A/R/U) ? '    
    wsadd['A2'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    wsadd['B2'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    wsadd['C2'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    wsadd['D2'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    wsadd['E2'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    wsadd['F2'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    wsadd['G2'].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)
    
    for col in range(1,8):
        wsadd[get_column_letter(col) + '2'].font = Font(bold=True ) 
        wsadd[get_column_letter(col) + '2'].alignment = Alignment(horizontal='center', vertical='bottom', wrap_text=True)
        wsadd[get_column_letter(col) + '2'].fill = PatternFill("solid", fgColor=HEADER_COLOR)
    
    wsadd.column_dimensions['A'].width = 30
    wsadd.column_dimensions['B'].width = 20
    wsadd.column_dimensions['C'].width = 10
    wsadd.column_dimensions['D'].width = 20
    wsadd.column_dimensions['E'].width = 20
    wsadd.column_dimensions['F'].width = 20   
    wsadd.column_dimensions['G'].width = 20       
    
    wsadd.row_dimensions[2].height = 60
 
   
    return work_book  


In [34]:
def create_sheet_feedback(work_book):
    
    # Create The Tab 
    ws_feedback = work_book.create_sheet("Feedback")
    ws_feedback.title = "Feedback or Issues" 
    ws_feedback.sheet_properties.tabColor = USER_TAB_COLOR
    
    # Blank the Canvas 
    fontStyle = Font(size = "12", bold = True)
    for col in range(1,25):
        for row in range(1,25):
            ws_feedback[get_column_letter(col) + str(row)].alignment = Alignment(horizontal='left', vertical='top', wrap_text=True)          
            ws_feedback[get_column_letter(col) + str(row)].font = fontStyle
            ws_feedback[get_column_letter(col) + str(row)].fill = PatternFill("solid", fgColor=BLANK_WHITE)
    
    
    # Create The Title 
    ws_feedback.merge_cells('A1:B1')
    ws_feedback.row_dimensions[1].height = 20
    top_left_cell = ws_feedback['A1']
    top_left_cell.value = "Feedback or Issues"

    top_left_cell.fill = PatternFill("solid", fgColor=USER_TAB_COLOR)
    top_left_cell.fill = fill = GradientFill(stop=(USER_TAB_COLOR, BLANK_WHITE))
    top_left_cell.font  = Font(b=True, color="000000")
    top_left_cell.alignment = Alignment(horizontal="center", vertical="center")
    fontStyle = Font(size = "18", bold = "True")
    top_left_cell.font = fontStyle

    ws_feedback['A2'] = 'User Name'
    ws_feedback['B2'] = 'Issue or Feedback'
  
    for col in range(1,3):
        ws_feedback[get_column_letter(col) + '2'].font = Font(size = "12", bold=True ) 
        ws_feedback[get_column_letter(col) + '2'].alignment = Alignment(horizontal='center', vertical='bottom', wrap_text=True)
        ws_feedback[get_column_letter(col) + '2'].fill = PatternFill("solid", fgColor=HEADER_COLOR)
    
    ws_feedback.column_dimensions['A'].width = 30
    ws_feedback.column_dimensions['B'].width = 120 

    ws_feedback.row_dimensions[2].height = 30
 
   
    return work_book  
 



In [35]:
# Automate the worksheet 
def create_work_book(work_book):
    create_sheet_one(wb)
    create_sheet_two(wb)
    create_sheet_three(wb)
    create_sheet_validation(wb) 
    create_instruction_sheet(wb)
    create_sheet_add(wb)  
    create_sheet_feedback(wb)      
    return work_book      

In [36]:
extension = 'xlsx'
os.chdir(Path_to_Excel)
files = glob.glob('*.{}'.format(extension))

for x in range(len(files)): 
    print(files[x]) 
    wb = load_workbook(Path_to_Excel + files[x])
    org_name =  files[x].replace('.xlsx','')
    create_work_book(wb)
    wb.save(Path_to_Save_Excel + files[x]) 


EC_PCP_Erie_County_Medical_Center_FM_Clinic.xlsx
EC_PCP_Erie_County_Medical_Center_Grider_Fam_Health_Clinic.xlsx
EC_PCP_Erie_County_Medical_Center_IM_Clinic.xlsx
EC_PCP_Erie_County_Medical_Center_VIP_Clinic.xlsx
EC_PCP_Erie_County_Medical_Center_You_Center_for_Wellness.xlsx
Erie_County_Medical_Center_Corporation.xlsx
Great_Lakes_Physician_PC.xlsx
GPPC_PCP_Amherst_Sheridan.xlsx
GPPC_PCP_Buffalo_Main.xlsx
GPPC_PCP_Dunkirk_Vineyard_Ste_1.xlsx
GPPC_PCP_East_Aurora_Main.xlsx
GPPC_PCP_Elma_Transit.xlsx
GPPC_PCP_Hamburg_Camp.xlsx
GPPC_PCP_Hamburg_Southwestern.xlsx
GPPC_PCP_Lancaster_Transit.xlsx
GPPC_PCP_Lockport_Olcott.xlsx
GPPC_PCP_Lockport_Snyder.xlsx
GPPC_PCP_Medina_Ohio.xlsx
GPPC_PCP_Orchard_Park_Big_Tree_Ste_107.xlsx
GPPC_PCP_Springville_South_Cascade.xlsx
GPPC_PCP_Tonawanda_Colvin.xlsx
GPPC_PCP_Willamsville_Maple.xlsx
GPPC_PCP_Williamsville_Youngs.xlsx
GPPC_Peds_Dunkirk_Vineyard_Ste_3.xlsx
GPPC_Post-Acute.xlsx
General_Physician_PC.xlsx
AA_PCP_Alan_M_Antfleck_Physician_PLLC_Delaware_Ave

In [ ]:
# Optional - Print out the compressive List of Patient Panels 
#Org_names = patient_panel.Organization_Name.unique()
#for org in Org_names:
#        print(org)

In [ ]:
# patient_panel.describe()

In [ ]:
# test examples 
#create_organization_patient_panel("GPPC PCP Springville South Cascade")
#create_organization_patient_panel("EC PCP Erie County Medical Center FM Clinic")


In [ ]:
#Provider_panel = patient_panel[(patient_panel.Provider_Name == "ADAMS, TIMOTHY")] 

In [ ]:
#   create_work_book(wb)    

In [ ]:
# patient_panel['Gender'].value_counts()

In [ ]:
# patient_panel['Organization_Name'].value_counts()

In [ ]:
# Organization_panel.head()

In [ ]:
#wb.security.workbookPassword = 'population health'
#wb.security.lockStructure = True

In [ ]:
#spread_name = '\\Users\josep\Documents\Kaleida\'+ Organization_panel
#Organization_panel.to_excel('\\Users\josep\Documents\Kaleida\GPPC PCP Springville South Cascade.xlsx')
#"C:\Users\josep\Documents\Kaleida\Validation\AA_PCP_Alan_M_Antfleck_Physician_PLLC_Delaware_Ave.xlsx"

In [ ]:
#patient_panel.rename(columns = {'patient_age_in_years':'Patient Age'}, inplace = True)
#patient_panel.rename(columns = {'days_test':'Patient Precise Age'}, inplace = True)
#patient_panel.rename(columns = {'organization_class':'Organization_Class'}, inplace = True)
#patient_panel.rename(columns = {'organization_name':'Organization_Name'}, inplace = True)
#patient_panel.rename(columns = {'provider_name':'Provider_Name'}, inplace = True)
#patient_panel.rename(columns = {'patient_name':'Patient_Name'}, inplace = True)
#patient_panel.rename(columns = {'birth_date':'Birth_Date'}, inplace = True)
#patient_panel.rename(columns = {'patient_age_in_years':'Age'}, inplace = True)
#patient_panel.rename(columns = {'days_test':'Precise_Age'}, inplace = True)
#patient_panel.rename(columns = {'gender':'Gender'}, inplace = True)
#patient_panel.rename(columns = {'days_test':'Precise Age'}, inplace = True)


In [ ]:
# Adjust Birth Date and Calculate Age  
#now = pd.Timestamp('now')
#patient_panel['Age_in_Years'] = now - patient_panel['Birth_Date']
#patient_panel['Age_in_Years'] = patient_panel['Age_in_Years']/np.timedelta64(1,'Y')
#patient_panel['Birth_Date'] = patient_panel['Birth_Date'].dt.date
#patient_panel.round({'Age_in_Years':1})